In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [2]:

import numpy as np
import os, sys
import h5py
import pandas as pd
import matplotlib.pyplot as plt

import readdata as rd

import datetime
sys.path.append('/Users/ajc/Desktop/ATS/ats-repo/ats/tools/utils/')
import parse_ats, transect_data

In [3]:
outfile = '/Users/ajc/AllPostProcessData/BarrowTransect/NGEE/Temperature-SiteC/'

In [4]:
infile = '/Users/ajc/Desktop/ATS/barrow-polygon-data/Vladimir-Data/Area_C/'
save = False

dataset1= 'NGEE_BRW_C_2012-09-09_2013-09-30_Thermal_Transect_and_Borehole.csv' # 2012-13
dataset2= 'NGEE_BRW_C_2013-10-01_2014-09-30_Thermal_Transect_and_Borehole.csv' # 2013-14
dataset3= 'NGEE_BRW_C_2014-10-01_2015-09-30_Thermal_Transect_and_Borehole.csv' # 2014-15


time_origin = datetime.datetime(2012,1,1,0,0,0)

infile_observed1 = infile + dataset1
infile_observed2 = infile + dataset2
infile_observed3 = infile + dataset3
data1 = pd.read_csv(infile_observed1, skiprows=5)
data2 = pd.read_csv(infile_observed2, skiprows=5)
data3 = pd.read_csv(infile_observed3, skiprows=5)

DataObserved2012 = dict()
DataObserved2013 = dict()
DataObserved2014 = dict()
for c in data1.columns:
    k = -100
    if 'trough' in c or 'center' in c or 'rim' in c:
        k = int(float(c[6:10])*100)
        d = np.array([ x + 273.15  for x in data1[c]])
    if 'trough' in c:
        DataObserved2012['soil_temp_trough_%scm'%k] = d
    elif 'center' in c and not 'off' in c :
        DataObserved2012['soil_temp_center_%scm'%k] = d
    elif 'rim' in c and 'rim2' not in c:
        DataObserved2012['soil_temp_rim_%scm'%k] = d
    if 'Time' in c:
        time1 = [pd.to_datetime(data1['Timestamp'])]
        time = [(t - time_origin).dt.total_seconds()/86400 for t in time1]
        Time = np.array([t for t in time[0]])
        DataObserved2012['time'] = np.array(Time)
#print len(DataObserved2012['time'])
for c in data2.columns:
    k = -100
    if 'trough' in c or 'center' in c or 'rim' in c:
        k = int(float(c[6:10])*100)
        d = np.array([ x + 273.15  for x in data2[c]])
    if 'trough' in c:
        DataObserved2013['soil_temp_trough_%scm'%k] = d
    elif 'center' in c and not 'off' in c :
        DataObserved2013['soil_temp_center_%scm'%k] = d
    elif 'rim' in c and 'rim2' not in c:
        if k != 2:
            k = k + 4
        DataObserved2013['soil_temp_rim_%scm'%k] = d
    if 'Time' in c:
        time1 = [pd.to_datetime(data2['Timestamp'])]
        time = [(t - time_origin).dt.total_seconds()/86400 for t in time1]
        Time = np.array([t for t in time[0]])
        DataObserved2013['time'] = np.array(Time)
    
Dapths2012 = dict()
Dapths2013 = dict()
Dapths2014 = dict()
for c in data3.columns:
    k = -100
    if 'trough' in c or 'center' in c or 'rim' in c:
        k = int(float(c[6:10])*100)
        d = np.array([ x + 273.15  for x in data3[c]])
    if 'trough' in c:
        DataObserved2014['soil_temp_trough_%scm'%k] = d
        #Dapths2012['soil_temp_trough_%scm'%k] = k
    elif 'center' in c and not 'off' in c :
        DataObserved2014['soil_temp_center_%scm'%k] = d
        #Dapths2013['soil_temp_trough_%scm'%k] = k
        #print 'soil_temp_center_%scm'%k
    elif 'rim' in c and 'rim2' not in c:
        if k != 2:
            k = k + 4
        #print 'soil_temp_rim_%scm'%k
        DataObserved2014['soil_temp_rim_%scm'%k] = d
        #Dapths2014['soil_temp_trough_%scm'%k] = k
    if 'Time' in c:
        time1 = [pd.to_datetime(data3['Timestamp'])]
        time = [(t - time_origin).dt.total_seconds()/86400 for t in time1]
        Time = np.array([t for t in time[0]])
        DataObserved2014['time'] = np.array(Time)


In [5]:
reload(transect_data)
def ReadTransectData(dirname, varnames=['temperature']):
    keys, times, dats = parse_ats.readATS(dirname, "visdump_surface_data.h5")
    matrix = transect_data.transect_data(varnames,directory=dirname,return_map=False)
    return times, matrix

In [210]:
simulation = 'SiteC_2R2'
infile_simulated = '/Users/ajc/Projects/ATS-Data/OR-CONDO/simulations/barrow_icewedges/NGEE/SiteC_March8/%s/'%simulation

DataSim = dict()
Time, data = ReadTransectData(infile_simulated)

DataSim['time_yr'] = np.array(Time)
DataSim['data_orig'] = np.array(data)
DataSim['time_d'] = np.array([t*365.25 -730. for t in Time])

In [211]:
"""
data[Arg1, Arg2, Arg3, Arg4]
lenght of Arg1 = nvar -1
Arg1 = 0 (x-axis), Arg1 =1 (z-axis), Arg1 = 2 or var -1 (soil temperature)
Arg2 = cycles (time steps)
Arg3 = 
data[1, 0, xpos,znum-1] 
"""


'\ndata[Arg1, Arg2, Arg3, Arg4]\nlenght of Arg1 = nvar -1\nArg1 = 0 (x-axis), Arg1 =1 (z-axis), Arg1 = 2 or var -1 (soil temperature)\nArg2 = cycles (time steps)\nArg3 = \ndata[1, 0, xpos,znum-1] \n'

In [212]:
def getTempAtDepth(location= 'center', depth=2.0):
    nvar, cycles, xnum, znum = DataSim['data_orig'].shape
    Temp = np.zeros(cycles)
    xpos, zpos = -1,-1
  
    if location == 'center':
        xpos = xnum - 1
    elif location == 'rim':
        xpos = 10
    elif location == 'trough':
        xpos = 0
    
    datum = data[1, 0, xpos,znum-1] 
    depth = datum - depth/100.
    
    for i in range(znum):
        z1 = data[1, 0, xpos,i]
        if z1 > depth:
            zpos = i
            #print 'depth', i, zpos, depth, data[1, 0, xpos,i]
            break
            
    assert xpos >= 0 and zpos >=0
    for i in range(cycles):
        Temp[i] = data[nvar-1, i, xpos,zpos]
    return Temp


depth = [2,5, 10, 15, 20, 25, 30, 35, 40, 50, 60, 70, 80, 100, 125, 150]

for d in depth:
    DataSim['soil_temp_center_%scm'%d] = getTempAtDepth(location='center', depth=d)
    DataSim['soil_temp_trough_%scm'%d] = getTempAtDepth(location='trough', depth=d)
    DataSim['soil_temp_rim_%scm'%d] = getTempAtDepth(location='rim', depth=d)


In [213]:
def put_axis(dep, loc=''):
    #return
    plt.xlabel('Day',fontsize=12, fontweight='normal')
    plt.ylabel('Temperature [K]',fontsize=12, fontweight='normal')
    #--plt.text(4, 285, '-%s, %s, %s'%(dep,start_year, loc),fontsize=12, fontweight='normal')
    plt.text(260, 280, 'depth = %s'%dep,fontsize=11, fontweight='normal')
    #plt.text(4, 278, '%s'%(loc),fontsize=12, fontweight='normal')
    plt.ylim((245, 285))
    plt.yticks(np.linspace(245,285, 5))
    plt.xlim((240, 1100))
    plt.xticks(np.linspace(240,1100, 5))
    #plt.axhline(y=273.25,color='b')

DataObserved2012Daily = dict()
DataObserved2013Daily = dict()
DataObserved2014Daily = dict()

L2012 = len(DataObserved2012['soil_temp_trough_2cm'])
R2012 = L2012%24
print R2012
for key, d in DataObserved2012.iteritems():
    DataObserved2012Daily[key] = np.reshape(DataObserved2012[key][:-R2012], (-1, 24)).mean(axis=1)

L2013 = len(DataObserved2013['soil_temp_trough_2cm'])
R2013 = L2013%24
print L2013, R2013
for key, d in DataObserved2013.iteritems():
    DataObserved2013Daily[key] = np.reshape(DataObserved2013[key][:-R2013], (-1, 24)).mean(axis=1)

L2014 = len(DataObserved2014['soil_temp_trough_2cm'])
R2014 = L2014%24
print R2014
for key, d in DataObserved2014.iteritems():
    DataObserved2014Daily[key] = np.reshape(DataObserved2014[key][:-R2014], (-1, 24)).mean(axis=1)


15
8759 23
19


In [214]:
outfile = '/Users/ajc/AllPostProcessData/2019/simulations/barrow-iwp/March8/Temp/'
def plotData(location = 'center', year = 2012):
    fig, axs = plt.subplots(3,3, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)

    plt.tight_layout()
    axs = axs.ravel()
    var = 'soil_temp_%s_'%location
    DataP = dict()
    if year == 2012:
        DataP = DataObserved2012Daily
    elif year == 2013:
        DataP = DataObserved2013Daily
    elif year == 2014:
        DataP = DataObserved2014Daily
    for key in DataP.keys():
        if 'rim' in key:
            print key
    plt.subplot(331)
    plt.plot(DataP['time'], DataP[var + '5cm'],'r-')
    plt.plot(DataSim['time_d'], DataSim[var + '5cm'],'k')
    put_axis('5cm', 'center')

    plt.subplot(332)
    plt.plot(DataP['time'], DataP[var + '10cm'],'r-')
    plt.plot(DataSim['time_d'], DataSim[var + '10cm'],'k')
    put_axis('10cm', 'center')

    plt.subplot(333)
    plt.plot(DataP['time'], DataP[var + '20cm'],'r-')
    plt.plot(DataSim['time_d'], DataSim[var + '20cm'],'k')
    put_axis('20cm', 'center')

    plt.subplot(334)
    plt.plot(DataP['time'], DataP[var + '40cm'],'r-')
    plt.plot(DataSim['time_d'], DataSim[var + '40cm'],'k')
    put_axis('40cm', 'center')

    plt.subplot(335)
    plt.plot(DataP['time'], DataP[var + '60cm'],'r-')
    plt.plot(DataSim['time_d'], DataSim[var + '60cm'],'k')
    put_axis('60cm', 'center')

    plt.subplot(336)
    plt.plot(DataP['time'], DataP[var + '80cm'],'r-')
    plt.plot(DataSim['time_d'], DataSim[var + '80cm'],'k')
    put_axis('80cm', 'center')

    plt.subplot(337)
    plt.plot(DataP['time'], DataP[var + '100cm'],'r-')
    plt.plot(DataSim['time_d'], DataSim[var + '100cm'],'k')
    put_axis('100cm', 'center')

    plt.subplot(338)
    plt.plot(DataP['time'], DataP[var + '125cm'],'r-')
    plt.plot(DataSim['time_d'], DataSim[var + '125cm'],'k')
    put_axis('125cm', 'center')

    plt.subplot(339)
    plt.plot(DataP['time'], DataP[var + '150cm'],'r',label='Observed')
    plt.plot(DataSim['time_d'], DataSim[var + '150cm'],'k', label='Simulated')
    put_axis('150cm', 'center')

    plt.legend(loc='upper right', fontsize=10, ncol=3, bbox_to_anchor=(0.7,- 0.32, .1, .1))

    plt.tight_layout()
    
    plt.savefig(outfile+'%s-temp-%s-%s.pdf'%(simulation, location,year), bbox_inches='tight', dpi=600)

In [215]:
#outfile = '/Users/ajc/AllPostProcessData/2019/simulations/barrow-iwp/Feb22/'
def plotDataAll(location = 'center'):
    fig, axs = plt.subplots(3,3, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)

    plt.tight_layout()
    axs = axs.ravel()
    var = 'soil_temp_%s_'%location
    
    plt.subplot(331)
    plt.plot(DataObserved2012Daily['time'],  DataObserved2012Daily[var + '10cm'],'r-')
    plt.plot(DataObserved2013Daily['time'],  DataObserved2013Daily[var + '10cm'],'r-')
    plt.plot(DataObserved2014Daily['time'],  DataObserved2014Daily[var + '10cm'],'r-')
    plt.plot(DataSim['time_d'], DataSim[var + '10cm'],'k')
    plt.axhline(y=273.15,linestyle='--',color='g')
    put_axis('10cm', 'center')
    
    plt.subplot(332)
    plt.plot(DataObserved2012Daily['time'],  DataObserved2012Daily[var + '15cm'],'r-')
    plt.plot(DataObserved2013Daily['time'],  DataObserved2013Daily[var + '15cm'],'r-')
    plt.plot(DataObserved2014Daily['time'],  DataObserved2014Daily[var + '15cm'],'r-')
    plt.plot(DataSim['time_d'], DataSim[var + '15cm'],'k')
    plt.axhline(y=273.15,linestyle='--',color='g')
    put_axis('15cm', 'center')

    plt.subplot(333)
    plt.plot(DataObserved2012Daily['time'],  DataObserved2012Daily[var + '20cm'],'r-')
    plt.plot(DataObserved2013Daily['time'],  DataObserved2013Daily[var + '20cm'],'r-')
    plt.plot(DataObserved2014Daily['time'],  DataObserved2014Daily[var + '20cm'],'r-')
    plt.axhline(y=273.15,linestyle='--',color='g')
    plt.plot(DataSim['time_d'], DataSim[var + '20cm'],'k')
    put_axis('20cm', 'center')
    
    plt.subplot(334)
    plt.plot(DataObserved2012Daily['time'],  DataObserved2012Daily[var + '40cm'],'r-')
    plt.plot(DataObserved2013Daily['time'],  DataObserved2013Daily[var + '40cm'],'r-')
    plt.plot(DataObserved2014Daily['time'],  DataObserved2014Daily[var + '40cm'],'r-')
    plt.plot(DataSim['time_d'], DataSim[var + '40cm'],'k')
    plt.axhline(y=273.15,linestyle='--',color='g')
    put_axis('40cm', 'center')

    plt.subplot(335)
    plt.plot(DataObserved2012Daily['time'],  DataObserved2012Daily[var + '60cm'],'r-')
    plt.plot(DataObserved2013Daily['time'],  DataObserved2013Daily[var + '60cm'],'r-')
    plt.plot(DataObserved2014Daily['time'],  DataObserved2014Daily[var + '60cm'],'r-')
    plt.plot(DataSim['time_d'], DataSim[var + '60cm'],'k')
    plt.axhline(y=273.15,linestyle='--',color='g')
    put_axis('60cm', 'center')

    plt.subplot(336)
    plt.plot(DataObserved2012Daily['time'],  DataObserved2012Daily[var + '80cm'],'r-')
    plt.plot(DataObserved2013Daily['time'],  DataObserved2013Daily[var + '80cm'],'r-')
    plt.plot(DataObserved2014Daily['time'],  DataObserved2014Daily[var + '80cm'],'r-')
    plt.plot(DataSim['time_d'], DataSim[var + '80cm'],'k')
    plt.axhline(y=273.15,linestyle='--',color='g')
    put_axis('80cm', 'center')

    plt.subplot(337)
    plt.plot(DataObserved2012Daily['time'],  DataObserved2012Daily[var + '100cm'],'r-')
    plt.plot(DataObserved2013Daily['time'],  DataObserved2013Daily[var + '100cm'],'r-')
    plt.plot(DataObserved2014Daily['time'],  DataObserved2014Daily[var + '100cm'],'r-')
    plt.plot(DataSim['time_d'], DataSim[var + '100cm'],'k')
    plt.axhline(y=273.15,linestyle='--',color='g')
    put_axis('100cm', 'center')

    plt.subplot(338)
    plt.plot(DataObserved2012Daily['time'],  DataObserved2012Daily[var + '125cm'],'r-')
    plt.plot(DataObserved2013Daily['time'],  DataObserved2013Daily[var + '125cm'],'r-')
    plt.plot(DataObserved2014Daily['time'],  DataObserved2014Daily[var + '125cm'],'r-')
    plt.plot(DataSim['time_d'], DataSim[var + '125cm'],'k')
    plt.axhline(y=273.15,linestyle='--',color='g')
    put_axis('125cm', 'center')

    plt.subplot(339)
    plt.plot(DataObserved2012Daily['time'],  DataObserved2012Daily[var + '150cm'],'r-')
    plt.plot(DataObserved2013Daily['time'],  DataObserved2013Daily[var + '150cm'],'r-')
    plt.plot(DataObserved2014Daily['time'],  DataObserved2014Daily[var + '150cm'],'r-',label='Observed')
    plt.plot(DataSim['time_d'], DataSim[var + '150cm'],'k', label='Simulated')
    plt.axhline(y=273.15,linestyle='--',color='g')
    put_axis('150cm', 'center')
    
    plt.legend(loc='upper right', fontsize=10, ncol=3, bbox_to_anchor=(0.7,- 0.32, .1, .1))

    plt.tight_layout()
    
    plt.savefig(outfile+'%s-temp-%s.pdf'%(simulation, location), bbox_inches='tight', dpi=600)

In [218]:
#plotData(location='rim',year=2013)
plotDataAll(location='center')

In [16]:

#smoothing data
import h5py
import numpy as np
infileDM = h5py.File('/Users/ajc/Desktop/ATS/forcing_data/barrow1985-2015/barrow1985-2015-trend.h5','r')
infileAnna = h5py.File('/Users/ajc/Desktop/ATS/forcing_data/Barrow-Anna-WaterLevel/Barrow-2010_2016-updated-smooth.h5','r')
infileAnna1 = h5py.File('/Users/ajc/Desktop/ATS/forcing_data/Barrow-Anna-WaterLevel/Barrow-2010_2016-updated.h5','r')
print infileAnna.keys()
TimeDM = [t/86400. for t in infileDM['time [s]']] 
TimeAnna = [(25*365*0+t/86400.) for t in infileAnna['time [s]']]

filesurf= infile_simulated + '/visdump_surface_data.h5'
data =rd.GetSurfVarFromVis(file=filesurf,var='surface-incoming_longwave_radiation',cellid=0)
data = [d for d in data['surface-incoming_longwave_radiation']]

#plt.plot(TimeDM, infileDM['air temperature [K]'], 'r')
plt.plot(TimeAnna, infileAnna['incoming longwave radiation [W m^-2]'], 'g',label='Anna\'s')
plt.plot(TimeDM, infileDM['incoming longwave radiation [W m^-2]'], 'c',label='Daymet')
plt.plot(TimeAnna[:1095],data,'k',label='simulated')
plt.xlabel('Day',fontsize=12, fontweight='normal')
plt.ylabel('incoming longwave radiation [W m^-2]',fontsize=12, fontweight='normal')
plt.legend()
plt.show()

[u'air temperature [K]', u'incoming longwave radiation [W m^-2]', u'incoming shortwave radiation [W m^-2]', u'precipitation rain [m s^-1]', u'precipitation snow [m SWE s^-1]', u'relative humidity [-]', u'time [s]', u'wind speed [m s^-1]']


In [ ]:
# Kr
import math
h_cut_u, h_cut_d= 0,10
uf = np.linspace(0,1,100)
h = 101334
Kr = []
for u in uf:
    if (h >=  101325. + h_cut_u):
        Kr.append(np.sin(np.pi * u/ 2.)**4)
    elif (h < 101325. + h_cut_d):
        print h
        Kr.append(1.)
    else:
        fac = (h - (101325 + h_cut_d)) / (h_cut_u - h_cut_d);
        k = (1-fac) + fac*(np.sin(np.pi * u/2.)**4);
        Kr.append(k)
plt.plot(uf,Kr,'r')
